In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
import os


def set_environment_variables(env_path: str = '../.env',):
    with open(env_path, 'r') as f:
        key, val = f.readline().split()
        os.environ[key] = val

In [3]:
set_environment_variables()

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat
import sys, os

# Авторизация в GigaChat
llm = GigaChat(
    credentials=os.environ['GIGACHAT_API_KEY'],
    scope='GIGACHAT_API_PERS',
    model="GigaChat",
    # Отключает проверку наличия сертификатов НУЦ Минцифры
    verify_ssl_certs=False,
    streaming=False,
)

messages = [
    SystemMessage(
        content="Ты эмпатичный бот-психолог, который помогает пользователю решить его проблемы."
    )
]

while(True):
    user_input = input("Пользователь: ")
    if user_input == "пока":
      break
    messages.append(HumanMessage(content=user_input))
    res = llm.invoke(messages)
    messages.append(res)
    print("GigaChat: ", res.content)

GigaChat:  Здравствуйте! Как я могу вам помочь?
GigaChat:  Здравствуйте! Как я могу вам помочь?
GigaChat:  Здравствуйте! Как я могу вам помочь?
GigaChat:  Мне очень жаль это слышать. Что случилось, что вам так плохо?
GigaChat:  Мне очень жаль это слышать. Что случилось, что вам так плохо?
GigaChat:  Мне очень жаль это слышать. Что случилось, что вам так плохо?


KeyboardInterrupt: Interrupted by user

In [4]:
import os, sys

print(os.environ.get("SHIT"))

None


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter 
from langchain.document_loaders import TextLoader

loader = TextLoader('../preprocessedDocs/habr.txt')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)
db

/tmp/ipykernel_3997/1456671600.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/sergey/Projects/BurnoutAnalysis/venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
retriever = db.as_retriever()
retrieved_docs = retriever.invoke("Я хочу умереть")

retrieved_docs[0].page_content

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


'О нем много говорят на него часто жалуются с ним пытаются бороться и путают его с обычной усталостью Скорее всего каждый из нас хоть раз с ним сталкивался но продолжал вкладывать последние силы в рабочий проект  \nМы с HR-платформой Beehive опросили 2000 IT-специалистов чтобы понять как обстоят дела с выгоранием в IT-сфере и провести качественное измерение этой как будто уже вечной проблемы\nА советы для специалистов и компаний оставили в самом конце статьи \nВыгорание  состояние эмоционального психологического и физического истощения которое часто развивается из-за долгого переживания хронического стресса на рабочем месте Оно может затронуть людей в самых разных сферах и должностях \nЭмоциональное выгорание характеризуется чрезмерной усталостью отсутствием энергии и ощущением эмоционального истощения\nДеперсонализация или апатия выражается в отстраненном циничном отношении к окружающим клиентам или коллегам\nПрофессиональная эффективность уменьшается что приводит к потере удовлетворе

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [15]:
llm = GigaChat(
    credentials=os.environ['GIGACHAT_API_KEY'],
    model="GigaChat",
    verify_ssl_certs=False,
    profanity_check=False,
)

prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:'''
)

In [16]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [17]:
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
retrieval_chain.invoke({'input': "Можешь рассказать про выгорание в IT-сфере?"})

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
Giga generation stopped with reason: blacklist


{'input': 'Можешь рассказать про выгорание в IT-сфере?',
 'context': [Document(metadata={'source': '../preprocessedDocs/habr.txt'}, page_content='О нем много говорят на него часто жалуются с ним пытаются бороться и путают его с обычной усталостью Скорее всего каждый из нас хоть раз с ним сталкивался но продолжал вкладывать последние силы в рабочий проект  \nМы с HR-платформой Beehive опросили 2000 IT-специалистов чтобы понять как обстоят дела с выгоранием в IT-сфере и провести качественное измерение этой как будто уже вечной проблемы\nА советы для специалистов и компаний оставили в самом конце статьи \nВыгорание  состояние эмоционального психологического и физического истощения которое часто развивается из-за долгого переживания хронического стресса на рабочем месте Оно может затронуть людей в самых разных сферах и должностях \nЭмоциональное выгорание характеризуется чрезмерной усталостью отсутствием энергии и ощущением эмоционального истощения\nДеперсонализация или апатия выражается в

In [12]:
len(docs[0].page_content)

11601

In [6]:
len(documents)

1

In [18]:
retriever.invoke("Я хочу питсы")

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


[Document(metadata={'source': '../preprocessedDocs/habr.txt'}, page_content='О нем много говорят на него часто жалуются с ним пытаются бороться и путают его с обычной усталостью Скорее всего каждый из нас хоть раз с ним сталкивался но продолжал вкладывать последние силы в рабочий проект  \nМы с HR-платформой Beehive опросили 2000 IT-специалистов чтобы понять как обстоят дела с выгоранием в IT-сфере и провести качественное измерение этой как будто уже вечной проблемы\nА советы для специалистов и компаний оставили в самом конце статьи \nВыгорание  состояние эмоционального психологического и физического истощения которое часто развивается из-за долгого переживания хронического стресса на рабочем месте Оно может затронуть людей в самых разных сферах и должностях \nЭмоциональное выгорание характеризуется чрезмерной усталостью отсутствием энергии и ощущением эмоционального истощения\nДеперсонализация или апатия выражается в отстраненном циничном отношении к окружающим клиентам или коллегам\n

Попробуем юзать два ретривера

In [24]:
import string


def tokenize(s):
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")


In [26]:
from langchain.chains import RetrievalQA

embedding_retriever = retriever
bm25_retriever = BM25Retriever.from_documents(
    documents=docs,
    preprocess_func=tokenize,
    k=3,
)

ensemble_retriever = EnsembleRetriever(
    retrievers=[embedding_retriever, bm25_retriever],
    weights=[0.4, 0.6],
)


qa = RetrievalQA.from_chain_type(
    llm=llm,
    hcain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

ValidationError: 1 validation error for RetrievalQA
hcain_type
  Extra inputs are not permitted [type=extra_forbidden, input_value='stuff', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden